In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

SyntaxError: ignored

In [0]:
#importing libraries

import numpy as np #for mathematical calculations if any needed 
import pandas as pd #for playing with dataframe
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer #For tokenizing the words
from keras.preprocessing.sequence import pad_sequences #for padding the words of same length
from keras.models import Sequential #for building a sequence models
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout #for defining a layers inside the model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import csv
import re #for manipulating with regex
import nltk #importing stopwords to be removed from the dataset

Using TensorFlow backend.


In [0]:
#importing the dataset
train_df = pd.read_csv("/content/drive/My Drive/Movie_Reviews/RNN/train_cleaned.csv")
test_df = pd.read_csv("/content/drive/My Drive/Movie_Reviews/RNN/test_cleaned.csv")

In [0]:
df = pd.concat([train_df,test_df],ignore_index = True)

In [0]:
df = df[0:1000]

In [0]:
#Cleaning the Train Dataset
#train_df['reviews'] = train_df['reviews'].apply(lambda x: re.sub('<br />',' ',x)) #removing nextline. If it is in another format, please check and remove it.
df['reviews'] = df['reviews'].apply(lambda x: re.sub('<br />',' ',x)) #removing nextline. If it is in another format, please check and remove it.
df['reviews'] = df['reviews'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
df['reviews'] = df['reviews'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
#df['reviews'] = df['reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) #removing stopwords

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['reviews'])
total_words = len(tokenizer.word_index) + 1

In [0]:
## convert data to sequence of tokens 
input_sequences = []
for line in df['reviews']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [0]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

923

In [0]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=10, padding='pre'))

In [0]:
predictors = input_sequences[:, :-1]

In [0]:
labels = input_sequences[:,-1]

In [0]:
labels = to_categorical(labels, num_classes=total_words)

In [0]:
#Declaring filepath for saving weigths
filepath = "weigths.best.hdf5"
#creating a checkpoint
checkpoint = ModelCheckpoint(filepath,verbose=1, mode = 'max')

In [0]:
input_len = 9
model = Sequential()
model.add(Embedding(total_words, 10, input_length=input_len))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
!ls

drive  sample_data


In [0]:
model.load_weights("weigths.best.hdf5")

OSError: ignored

In [0]:
model.fit(predictors, labels, epochs=200, verbose=1, batch_size = 200, callbacks=[checkpoint])

Epoch 1/200
115581/115581 [==============================] - 30s 262us/step - loss: 2.1108

Epoch 00001: saving model to weigths.best.hdf5
Epoch 2/200
115581/115581 [==============================] - 29s 247us/step - loss: 2.1078

Epoch 00002: saving model to weigths.best.hdf5
Epoch 3/200
115581/115581 [==============================] - 28s 243us/step - loss: 2.1029

Epoch 00003: saving model to weigths.best.hdf5
Epoch 4/200
115581/115581 [==============================] - 28s 245us/step - loss: 2.0996

Epoch 00004: saving model to weigths.best.hdf5
Epoch 5/200
115581/115581 [==============================] - 28s 243us/step - loss: 2.1002

Epoch 00005: saving model to weigths.best.hdf5
Epoch 6/200
115581/115581 [==============================] - 28s 244us/step - loss: 2.0981

Epoch 00006: saving model to weigths.best.hdf5
Epoch 7/200
115581/115581 [==============================] - 28s 245us/step - loss: 2.0913

Epoch 00007: saving model to weigths.best.hdf5
Epoch 8/200
115581/115581 [

In [0]:
def generate_text(starting_words, num_words, seq_length, model):
  for j in range(num_words):
    print(starting_words)
    token_list = tokenizer.texts_to_sequences([starting_words])[0]
    token_list = pad_sequences([token_list], maxlen=9, padding='pre')
    predicted = model.predict_classes(token_list,verbose=0)
    output_word = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(predicted)]
    starting_words += (output_word + " ")
    
  return starting_words
    

In [0]:
seq_length = 10
num_words = 10
generate_text("which ", num_words, seq_length, model)

NameError: ignored

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 9, 10)             162960    
_________________________________________________________________
lstm_9 (LSTM)                (None, 150)               96600     
_________________________________________________________________
dropout_9 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 16296)             2460696   
Total params: 2,720,256
Trainable params: 2,720,256
Non-trainable params: 0
_________________________________________________________________
